In [ ]:
import zipfile
from google.colab import files

z = zipfile.ZipFile('/content/drive/MyDrive/Data/cover.zip','r')
z.extractall()
z = zipfile.ZipFile('/content/drive/MyDrive/Data/stego.zip','r')
z.extractall()

import numpy as np
import cv2
import random
from glob import glob
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F

In [ ]:
BATCH_SIZE = 16
MOMENTUM = 0.95
LEARNING_RATE = 0.01
EPOCHS = 300
DECEY_STEP = 30          
WEIGHT_DECEY = 0.0001 
MAIN_DIR = '/content'

In [ ]:
hpf_list = [np.array([[0, 0, 0, 0, 0],
                         [0, 1, 0, 0, 0],
                         [0, 0, -1, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 0],
                         [0, 0, -1, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 1, 0],
                         [0, 0, -1, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, -1, 1, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, -1, 0, 0],
                         [0, 0, 0, 1, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, -1, 0, 0],
                         [0, 0, 1, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, -1, 0, 0],
                         [0, 1, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 1, -1, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 1, 0, 0, 0],
                         [0, 0, -2, 0, 0],
                         [0, 0, 0, 1, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 0],
                         [0, 0, -2, 0, 0],
                         [0, 0, 1, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 1, 0],
                         [0, 0, -2, 0, 0],
                         [0, 1, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 1, -2, 1, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[-1, 0, 0, 0, 0],
                         [0, 3, 0, 0, 0],
                         [0, 0, -3, 0, 0],
                         [0, 0, 0, 1, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, -1, 0, 0],
                         [0, 0, 3, 0, 0],
                         [0, 0, -3, 0, 0],
                         [0, 0, 1, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, -1],
                         [0, 0, 0, 3, 0],
                         [0, 0, -3, 0, 0],
                         [0, 1, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 1, -3, 3, -1],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 1, 0, 0, 0],
                         [0, 0, -3, 0, 0],
                         [0, 0, 0, 3, 0],
                         [0, 0, 0, 0, -1]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 0],
                         [0, 0, -3, 0, 0],
                         [0, 0, 3, 0, 0],
                         [0, 0, -1, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 1, 0],
                         [0, 0, -3, 0, 0],
                         [0, 3, 0, 0, 0],
                         [-1, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [-1, 3, -3, 1, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, -1, 2, -1, 0],
                         [0, 2, -4, 2, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 2, -1, 0],
                         [0, 0, -4, 2, 0],
                         [0, 0, 2, -1, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 2, -4, 2, 0],
                         [0, -1, 2, -1, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, -1, 2, 0, 0],
                         [0, 2, -4, 0, 0],
                         [0, -1, 2, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[-1, 2, -2, 2, -1],
                         [2, -6, 8, -6, 2],
                         [-2, 8, -12, 8, -2],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[0, 0, -2, 2, -1],
                         [0, 0, 8, -6, 2],
                         [0, 0, -12, 8, -2],
                         [0, 0, 8, -6, 2],
                         [0, 0, -2, 2, -1]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [-2, 8, -12, 8, -2],
                         [2, -6, 8, -6, 2],
                         [-1, 2, -2, 2, -1]], dtype=np.float32),
               np.array([[-1, 2, -2, 0, 0],
                         [2, -6, 8, 0, 0],
                         [-2, 8, -12, 0, 0],
                         [2, -6, 8, 0, 0],
                         [-1, 2, -2, 0, 0]], dtype=np.float32),
               np.array([[0, 0, 0, 0, 0],
                         [0, -1, 2, -1, 0],
                         [0, 2, -4, 2, 0],
                         [0, -1, 2, -1, 0],
                         [0, 0, 0, 0, 0]], dtype=np.float32),
               np.array([[-1, 2, -2, 2, -1],
                         [2, -6, 8, -6, 2],
                         [-2, 8, -12, 8, -2],
                         [2, -6, 8, -6, 2],
                         [-1, 2, -2, 2, -1]], dtype=np.float32)
               ]

In [ ]:
class PrePro(nn.Module):
    def __init__(self):
        super(PrePro,self).__init__()
        self.srm = nn.Conv2d(in_channels = 1, out_channels = 30, kernel_size = 5, padding = 2, bias = False)
    
        hpf_weight = nn.Parameter(torch.Tensor(hpf_list).view(30, 1, 5, 5), requires_grad=False)
        self.srm.weight = hpf_weight
    
    def forward(self,input):
        output = self.srm(input)
        return output

class ABS(nn.Module):
    def __init__(self):
        super(ABS, self).__init__()

    def forward(self, input):
        output = torch.abs(input)
        return output

class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
        
    def forward(self, x):
        assert len(x.size()) == 4, x.size()
        B, C, W, H = x.size()
        return F.avg_pool2d(x, (W, H)).view(B, C)

class Trunc(nn.Module):
    def __init__(self,T):
        super(Trunc, self).__init__()
        self.trunc = T
  
    def forward(self, input):
        output = torch.clamp(input, min=-self.trunc, max=self.trunc)
        return output

class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.prepro = PrePro()
    
        self.conv1 = nn.Conv2d(in_channels = 30, out_channels = 30, kernel_size = 5, padding = 2)
        self.abs1 = ABS()
        self.bt1 = nn.BatchNorm2d(30)
        self.tr1 = Trunc(3.0)
    
        self.conv2 = nn.Conv2d(in_channels = 30, out_channels = 30, kernel_size = 5, padding = 2)
        self.bt2 = nn.BatchNorm2d(30)
        self.tr2 = Trunc(2.0)
        self.pool2 = nn.AvgPool2d(kernel_size = 5, stride = 2, padding = 2)

        self.conv3 = nn.Conv2d(in_channels = 30, out_channels = 32, kernel_size = 3, padding = 1)
        self.bt3 = nn.BatchNorm2d(32)
        self.rl3 = nn.ReLU()
        self.pool3 = nn.AvgPool2d(kernel_size = 5, stride = 2, padding = 2)

        self.conv4 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1)
        self.bt4 = nn.BatchNorm2d(64)
        self.rl4 = nn.ReLU()
        self.pool4 = nn.AvgPool2d(kernel_size = 5, stride = 2, padding = 2)

        self.conv5 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1)
        self.bt5 = nn.BatchNorm2d(128)
        self.rl5 = nn.ReLU()
        self.gap5 = GlobalAvgPool2d()

        self.li1 = nn.Linear(in_features = 128, out_features = 256)
        self.li2 = nn.Linear(in_features = 256, out_features = 1024)
        self.li3 = nn.Linear(in_features = 1024, out_features = 2)

    def forward(self,input):
        output = self.prepro(input)
        output = self.conv1(output)
        output = self.abs1(output)
        output = self.bt1(output)
        output = self.tr1(output)    

        output = self.conv2(output)
        output = self.bt2(output)
        output = self.tr2(output)
        output = self.pool2(output)

        output = self.conv3(output)
        output = self.bt3(output)
        output = self.rl3(output)
        output = self.pool3(output)

        output = self.conv4(output)
        output = self.bt4(output)
        output = self.rl4(output)
        output = self.pool4(output)

        output = self.conv5(output)
        output = self.bt5(output)
        output = self.rl5(output)
        output = self.gap5(output)

        output = F.relu(self.li1(output))
        output = F.relu(self.li2(output))
        output = self.li3(output)

        return output


In [ ]:
class TensorTransform():
    def  __call__(self,sample):
        data, label = sample['data'], sample['label']

        data = np.expand_dims(data, axis=1)
        data = data.astype(np.float32)

        data = torch.from_numpy(data)
        label = torch.from_numpy(label).long()

        new_sample = {'data': data ,'label': label}

        return new_sample

class CoverStegoDataset(Dataset):
    def __init__(self, data_list, transform = None):
        self.transform = transform

        self.data_list = data_list

        self.cover_dir = MAIN_DIR + '/cover'
        self.stego_dir = MAIN_DIR + '/stego'

    def __len__(self):
        return len(self.data_list)
  
    def __getitem__(self,index):
        image = self.data_list[index]

        cover_path_image = self.cover_dir + '/' + str(image) + '.pgm'
        stego_path_image = self.stego_dir + '/' + str(image) + '.pgm' 

        data_cover = cv2.imread(cover_path_image, -1)
        data_stego = cv2.imread(stego_path_image, -1)

        data = np.stack([data_cover, data_stego])
        label = np.array([0, 1], dtype='int32')

        sample = {'data': data, 'label': label}

        if self.transform:
            sample = self.transform(sample)
      
        return sample 

    def num_batches(self):
        return len(self.data_list) // BATCH_SIZE

def split_data():
    random.seed(42)
    data_list =[i+1 for i in range(40000)]
  
    random.shuffle(data_list)

    train_list = data_list[:20000]
    valid_list = data_list[20000:24000]
    test_list = data_list[24000:]

    return train_list, valid_list, test_list

def init_weights(m):
    if type(m) == nn.Conv2d:
        if m.weight.requires_grad:
            nn.init.xavier_normal_(m.weight)

    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)

In [ ]:
def train(model, device, train_loader, optimizer, loss_func, epoch, train_bar):

    loss_sum = 0.0
    running_loss = 0.0
    model.train()

    for i, sample in enumerate(train_loader):
        data, label = sample['data'], sample['label']

        shape = [*data.shape]
        data = data.reshape(shape[0] * shape[1], *shape[2:])
        label = label.reshape(-1)
    
        data, label = data.to(device), label.to(device)

        optimizer.zero_grad()

        output = model(data)
    
        loss = loss_func(output, label)
        loss_it = loss.item()
        loss_sum += loss_it * data.size(0)
        loss_avg = loss_sum / ((i+1)*data.size(0))

        loss.backward()

        optimizer.step()

        train_bar.set_postfix(loss = loss_avg, epoch = epoch)
        train_bar.update()

In [ ]:
def evaluate(model, device, valid_load, optimizer, epoch, valid_bar, best_acc):
  
    test_loss = 0.0
    correct = 0.0

    with torch.no_grad():
        for i, sample in enumerate(valid_load):
            data, label = sample['data'], sample['label']

            shape = [*data.shape]
            data = data.reshape(shape[0] * shape[1], *shape[2:])
            label = label.reshape(-1)

            data, label = data.to(device), label.to(device)

            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(label.view_as(pred)).sum().item()

            accuracy = correct/(len(valid_load.dataset) * 2)
      
            valid_bar.set_postfix(acc = accuracy, epoch = epoch)
            valid_bar.update()

    if accuracy > best_acc and epoch > 180:
        best_acc = accuracy

    return best_acc

def final_evaluate(model, device, test_load, optimizer, best_acc):
  
    test_loss = 0.0
    correct = 0.0

    with torch.no_grad():
        for i, sample in enumerate(test_load):
            data, label = sample['data'], sample['label']

            shape = [*data.shape]
            data = data.reshape(shape[0] * shape[1], *shape[2:])
            label = label.reshape(-1)

            data, label = data.to(device), label.to(device)

            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(label.view_as(pred)).sum().item()

            accuracy = correct/(len(test_load.dataset) * 2)

    if accuracy > best_acc:
        best_acc = accuracy

    return best_acc

In [ ]:
if __name__ == "__main__":
    
    device = torch.device("cuda")

    transformer = TensorTransform()

    train_list, valid_list, test_list = split_data()

    train_data = CoverStegoDataset(train_list, transformer)
    valid_data = CoverStegoDataset(valid_list, transformer)
    test_data = CoverStegoDataset(test_list, transformer)

    train_load = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle=True)
    valid_load = DataLoader(valid_data, batch_size = BATCH_SIZE, shuffle=True)
    test_load = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle=False)

    model = Network().to(device)
    model.apply(init_weights)

    params = model.parameters()

    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.SGD(params, lr = LEARNING_RATE, momentum = MOMENTUM, weight_decay = MOMENTUM)
    sheduler = optim.lr_scheduler.StepLR(optimizer, step_size = DECEY_STEP)

    startEpoch = 1
    best_acc = 0.0

    epoch_bar = tqdm(total = EPOCHS, desc = 'epoch', position = 1)
    train_bar = tqdm(total = train_data.num_batches() , desc = 'train', position = 1)
    valid_bar = tqdm(total = valid_data.num_batches(), desc = 'valid', position = 1)

    for epoch in range(startEpoch, EPOCHS + 1):
        train(model, device, train_load, optimizer, loss_func, epoch, train_bar)
        best_acc = evaluate(model, device, valid_load, optimizer, epoch, valid_bar, best_acc) 
        sheduler.step()

        epoch_bar.set_postfix(epoch = epoch)
        train_bar.n = 0
        valid_bar.n = 0
        epoch_bar.update()

    best_acc = 0.0
    result = final_evaluate(model, device, test_load, optimizer, best_acc)
    print(result)